In [ ]:
import pickle
import os
import numpy as np

In [ ]:
base_dirs = ['/localdata/juan/local/',
             '/localdata/juan/erehwon//',
             '/localdata/juan/numenor/']
experiments = ['pilco_ssgp_rbfp_1',
               'mcpilco_dropoutd_rbfp_3',
               'mcpilco_dropoutd_mlpp_4',
               'mcpilco_lndropoutd_rbfp_5',
               'mcpilco_lndropoutd_mlpp_6',
               'mcpilco_dropoutd_dropoutp_7',
               'mcpilco_lndropoutd_dropoutp_8']
result_files = []
for b in base_dirs:
    dirs = os.listdir(b)
    for e in experiments:
        for d in dirs:
            if d.find(e) == 0:
                res_dir = os.path.join(b,d)
                res_file = os.path.join(res_dir, 'results_50_10')
                result_files.append(res_file)
                res_file = os.path.join(res_dir, 'results_50_20')
                result_files.append(res_file)
print(result_files)

In [ ]:
base_dirs = ['/localdata/juan/inferno/',
             '/localdata/juan/rhys/',
             '/localdata/juan/numenor/'
             ]
experiments = ['pilco_ssgp_rbfp_1',
               'mcpilco_dropoutd_rbfp_3',
               'mcpilco_dropoutd_mlpp_4',
               'mcpilco_lndropoutd_rbfp_5',
               'mcpilco_lndropoutd_mlpp_6',
               'mcpilco_dropoutd_dropoutp_7',
               'mcpilco_lndropoutd_dropoutp_8']

result_files = []
for b in base_dirs:
    for e in experiments:
        res_dir = os.path.join(b,e)
        res_file = os.path.join(res_dir, 'results_50_10')
        result_files.append(res_file)
print(result_files)

In [ ]:
from collections import OrderedDict
result_arrays = OrderedDict()
for rpath in result_files:
    if not os.path.isfile(rpath):
        continue
    with open(rpath, 'rb') as f:
        print('Opening %s' % rpath)
        exp_type = None
        for e in experiments:
            if rpath.find(e) >= 0:
                exp_type = e
                break
        arrays = result_arrays.get(exp_type, [])
        arrays.append(pickle.load(f))
        result_arrays[exp_type] = arrays

In [ ]:
ids = [0,1,2,4,3,5,6]
names = ['SSGP-DYN_RBF-POL (PILCO)', 'DROPOUT-DYN_RBF-POL', 'DROPOUT-DYN_MLP-POL',
         'LOGNORMAL-DYN_RBF-POL', 'DROPOUT-DYN_DROPOUT-POL', 'LOGNORMAL-DYN_MLP-POL',
         'LOGNORMAL-DYN_DROPOUT-POL']

In [ ]:
from collections import OrderedDict
# gather all costs
costs = OrderedDict()
for e in experiments:
    exp_results = result_arrays[e]
    costs_e = []
    for results in exp_results:
        costs_i = []
        #learning_iter
        for rj in results: 
            costs_ij = []
            #trial
            for r in rj:
                costs_ij.append(r[2])
            costs_i.append(costs_ij)
        if len(costs_i) > 0 :
            costs_e.append(costs_i)
    costs_i = np.concatenate(costs_e, axis=1).squeeze()
    costs_i = costs_i/costs_i.shape[-1]
    print costs_i.shape
    mean_sum_costs = costs_i.sum(-1).mean(-1)
    std_sum_costs = costs_i.sum(-1).std(-1)
    costs[e] = (mean_sum_costs, std_sum_costs)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
exp1 = ['mcpilco_dropoutd_mlpp_4',
 'mcpilco_lndropoutd_mlpp_6',
 'mcpilco_dropoutd_dropoutp_7',
 'mcpilco_lndropoutd_dropoutp_8']
names = ['Binary Drop (p=0.1) Dyn, MLP Pol', 'Log-Normal Drop Dyn, MLP Pol', 'Binary Drop (p=0.1) Dyn, Drop MLP Pol p=0.1', 'Log-Normal Drop Dyn, Drop MLP Pol p=0.1']
linetype = ['--','-','--','-']
names = dict(zip(exp1 ,names))
linetype = dict(zip(exp1 ,linetype))
matplotlib.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(15,9))
t = range(len(costs.values()[0][1]))
for e in exp1:
    mean, std = costs[e]
    min_ = mean
    std_ = std
    #min_ = np.minimum.accumulate(mean)
    #std_ = std[np.array([np.where(mean==mi) for mi in min_]).flatten()]
    if names[e].find('rbf') < 0:
        pl, = plt.plot(t, min_, linetype[e], label=names[e], linewidth=2)
        #pl, = plt.plot(t, max_, linetype[e], label=names[e], linewidth=2, color=pl.get_color())
        alpha = 0.5
        for i in range(1,2):
            alpha = alpha*0.5
            lower_bound = min_ - i*std_*0.5
            #lower_bound = min_
            upper_bound = min_ + i*std_*0.5
            #upper_bound = max_
            plt.plot(t, upper_bound, linetype[e], linewidth=2,color=pl.get_color(),alpha=alpha)
            plt.plot(t, lower_bound, linetype[e], linewidth=2,color=pl.get_color(),alpha=alpha)
            plt.fill_between(t, lower_bound, upper_bound, alpha=alpha, color=pl.get_color(),  linestyle=linetype[e])
plt.legend()
plt.xlabel('Number of interactions (2.5 secs at 10 Hz each)')
plt.ylabel('Average cost (over 30 runs)')
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
exp1 = ['pilco_ssgp_rbfp_1',
 'mcpilco_dropoutd_rbfp_3',
 'mcpilco_lndropoutd_rbfp_5',]
names = ['SSGP Dyn, RBF Pol', 'Binary Drop (p=0.1) Dyn, RBF Pol', 'Log-Normal Drop Dyn, RBF Pol']
linetype = ['--','-','--','-']
names = dict(zip(exp1 ,names))
linetype = dict(zip(exp1 ,linetype))
matplotlib.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(15,9))
t = range(len(costs.values()[0][1]))
for e in exp1:
    mean, std = costs[e]
    min_, std_ = mean, std
    #min_ = np.minimum.accumulate(mean)
    #std_ = std[np.array([np.where(mean==mi) for mi in min_]).flatten()]
    if names[e].find('rbf') < 0:
        pl, = plt.plot(t, min_, linetype[e], label=names[e], linewidth=2)
        #pl, = plt.plot(t, max_, linetype[e], label=names[e], linewidth=2, color=pl.get_color())
        alpha = 0.5
        for i in range(1,2):
            alpha = alpha*0.5
            lower_bound = min_ - i*std_*0.5
            #lower_bound = min_
            upper_bound = min_ + i*std_*0.5
            #upper_bound = max_
            plt.plot(t, upper_bound, linetype[e], linewidth=2,color=pl.get_color(),alpha=alpha)
            plt.plot(t, lower_bound, linetype[e], linewidth=2,color=pl.get_color(),alpha=alpha)
            plt.fill_between(t, lower_bound, upper_bound, alpha=alpha, color=pl.get_color(),  linestyle=linetype[e])
plt.legend()
plt.xlabel('Number of interactions (2.5 secs at 10 Hz each)')
plt.ylabel('Average cost (over 30 runs)')
plt.show()